# Replicating Education Crowdfunding Results


### What is this?
This notebook provides step-by-step instructions for replicating the results from our recent study of fairness-accuracy trade-offs on the Education Crowdfunding dataset.

### What has already been done?
To make reproducing the bias analysis easier as well as provide more visibility into our results, this notebook starts off from a point where the public DonorsChoose dataset has already been injested into a postgres database and a grid of models (described in Supplementary Table 2 of our study) has already been run. We've made available a database dump with both the underlying data and the results of our model runs (see below for instructions on getting it set up).

### What if I want to re-run the models from scratch?
You can also re-run (or add to) the model grid starting from the public DonorsChoose data using our open-source machine learning pipeline `triage`, which makes it easier to run any model with a `scikit-learn`-style interface over temporal validation sets in a "top k" setting. See this repo's readme file for some tips on getting set up as well as pointers to a repo with an example of running triage using these data.


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
import yaml
import sqlalchemy

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None

from IPython.display import display 
from itertools import permutations
from jinja2 import Template
import dateparser


from ohio.ext.numpy import pg_copy_to_table

## Getting Set Up

We've provided an extract of the data in a postgres dump file (postgres 11.10), available at: 

https://dsapp-public-data-migrated.s3.amazonaws.com/education_crowdfunding_replication.dmp 

(NOTE: this file is about 16 GB compressed, so may take some time to download depending on your connection). You can load the data into your own postgres server using (you'll need to create the `education_crowdfunding` database first and fill in the host, port, and user below):

```bash
pg_restore -h {POSTGRES_HOST} -p {POSTGRES_PORT} -d education_crowdfunding -U {POSTGRES_USER} -O -j 8 education_crowdfunding_replication.dmp
```

Note that the dump file is compressed and you'll need a server with around 500 GB of free disk space.

Loading this file will populate your database with several schemas:
- **public**: The raw data from donors choose as well as some tables with calculated features and intermediate modeling tables
- **model_metadata**: Information about the models we ran, such as model types and hyperparameters (models were run with `triage`, which generates this schema. In `triage` a "model group" specifies a type of model and associated hyperparameter values, while a "model" is an instantiation of a given model group on a specific temporal validation split). Note that this schema contains information on other model runs with this dataset, in addition to the run used for the current study of fairness-accuracy trade-offs.
- **test_results**: Validation set statistics and predictions for the models. Here, `test_results.predictions` contains project-level predicted scores from each model in the grid, while `test_results.evaluations` contains aggregated summary statistics for each model.
- **train_results**: Training set statistics for the models, including feature importances.
- **features**: Intermediate tables containing calculated features from the `triage` run.
- **bias_working**: Intermediate tables from the bias analysis, as well as the mapping table between projects and school poverty levels, `bias_working.entity_demos`.
- **bias_results_submitted**: Results of the fairness-accuracy trade-offs from the study as submitted (see below to use these to replicate the figures from the study).
- **bias_results**: Empty bias analysis results tables that will be populated by re-running the fairness adjustments (see below for instructions).


Finally, to connect to the database, you'll need a yaml file named `db_profile.yaml` in the same directory as this notebook with your connection info:
```yaml
host: {POSTGRES_HOST}
user: {POSTGRES_USER}
db: education_crowdfunding
pass: {POSTGRES_PASSWORD}
port: {POSTGRES_PORT}
```


In [ ]:
with open('db_profile.yaml') as fd:
    config = yaml.full_load(fd)
    dburl = sqlalchemy.engine.url.URL(
        "postgresql",
        host=config["host"],
        username=config["user"],
        database=config["db"],
        password=config["pass"],
        port=config["port"],
    )
    engine_donors = sqlalchemy.create_engine(dburl, poolclass=sqlalchemy.pool.QueuePool)

Next, we define a helper function that will pull the results of the various strategies explored in the study. The table `model_adjustment_results_plevel` contains performance statistics for each model type we ran with and without adjusting for recall equity and `composite_results_plevel` contains performance statistics for a stategy that selects the best-performing model for each subgroup separately (we'll look at generating these tables from the raw modeling results as well below). The query below uses four different strategies to choose a model for each temporal validation split based on these performance statistics:

In [ ]:
def get_bias_results(engine, schema='bias_results'):
    sql = """
    WITH mg_rns AS (
      SELECT *,
             row_number() OVER (PARTITION BY train_end_time, list_size, metric, parameter ORDER BY base_value DESC, base_max_recall_ratio ASC, RANDOM()) AS rn_base,
             row_number() OVER (PARTITION BY train_end_time, list_size, metric, parameter ORDER BY adj_value DESC, adj_max_recall_ratio ASC, RANDOM()) AS rn_adj, 
             row_number() OVER (PARTITION BY train_end_time, list_size, metric, parameter ORDER BY adj_value DESC, adj_max_recall_ratio ASC, RANDOM()) AS rn_multi_adj
      FROM {schema}.model_adjustment_results_plevel
      WHERE past_train_end_time = train_end_time
    )
    , base_mgs AS (
      SELECT * FROM mg_rns WHERE rn_base = 1
    )
    , adj_mgs AS (
      SELECT * FROM mg_rns WHERE rn_adj = 1
    )
    , multi_adj_mgs AS (
      SELECT * FROM mg_rns WHERE rn_adj = 1
    )
    -- Simple model selection on last time period, baseline with no recall adjustments
    SELECT 'Best Unadjusted Metric - Unadjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.base_value AS value,
           r.base_max_recall_ratio AS max_recall_ratio,
           r.base_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.base_frac_not_highest AS frac_not_highest,
           r.base_frac_highest AS frac_highest,
           r.base_min_score_not_highest AS min_score_not_highest, 
           r.base_min_score_highest AS min_score_highest, 
           r.base_mean_score_not_highest AS mean_score_not_highest, 
           r.base_mean_score_highest AS mean_score_highest,
           r.base_precision_not_highest AS precision_not_highest, 
           r.base_precision_highest AS precision_highest, 
           r.base_prevalence_not_highest AS prevalence_not_highest, 
           r.base_prevalence_highest AS prevalence_highest, 
           r.base_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.base_positive_prevalence_highest AS positive_prevalence_highest,            
           r.base_positive_rate_not_highest AS positive_rate_not_highest, 
           r.base_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN base_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time

    UNION ALL

    -- Model selection on last time before adjustment, with adjustment applied
    SELECT 'Best Unadjusted Metric - Adjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.adj_value AS value,
           r.adj_max_recall_ratio AS max_recall_ratio,
           r.adj_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.adj_frac_not_highest AS frac_not_highest,
           r.adj_frac_highest AS frac_highest, 
           r.adj_min_score_not_highest AS min_score_not_highest, 
           r.adj_min_score_highest AS min_score_highest, 
           r.adj_mean_score_not_highest AS mean_score_not_highest, 
           r.adj_mean_score_highest AS mean_score_highest, 
           r.adj_precision_not_highest AS precision_not_highest, 
           r.adj_precision_highest AS precision_highest, 
           r.adj_prevalence_not_highest AS prevalence_not_highest, 
           r.adj_prevalence_highest AS prevalence_highest, 
           r.adj_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.adj_positive_prevalence_highest AS positive_prevalence_highest, 
           r.adj_positive_rate_not_highest AS positive_rate_not_highest, 
           r.adj_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN base_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time

    UNION ALL

    -- Model selection on last time after adjustment, with adjustment applied
    SELECT 'Best Adjusted Metric - Adjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.adj_value AS value,
           r.adj_max_recall_ratio AS max_recall_ratio,
           r.adj_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.adj_frac_not_highest AS frac_not_highest,
           r.adj_frac_highest AS frac_highest, 
           r.adj_min_score_not_highest AS min_score_not_highest, 
           r.adj_min_score_highest AS min_score_highest, 
           r.adj_mean_score_not_highest AS mean_score_not_highest, 
           r.adj_mean_score_highest AS mean_score_highest, 
           r.adj_precision_not_highest AS precision_not_highest, 
           r.adj_precision_highest AS precision_highest, 
           r.adj_prevalence_not_highest AS prevalence_not_highest, 
           r.adj_prevalence_highest AS prevalence_highest, 
           r.adj_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.adj_positive_prevalence_highest AS positive_prevalence_highest, 
           r.adj_positive_rate_not_highest AS positive_rate_not_highest, 
           r.adj_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN adj_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time
    
    UNION ALL

    -- Model selection on last time before multiple adjustment, with multiple adjustment applied
    SELECT 'Best Unadjusted Metric - MultiAdjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.multi_adj_value AS value,
           r.multi_adj_max_recall_ratio AS max_recall_ratio,
           r.multi_adj_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.multi_adj_frac_not_highest AS frac_not_highest,
           r.multi_adj_frac_highest AS frac_highest, 
           r.multi_adj_min_score_not_highest AS min_score_not_highest, 
           r.multi_adj_min_score_highest AS min_score_highest, 
           r.multi_adj_mean_score_not_highest AS mean_score_not_highest, 
           r.multi_adj_mean_score_highest AS mean_score_highest, 
           r.multi_adj_precision_not_highest AS precision_not_highest, 
           r.multi_adj_precision_highest AS precision_highest, 
           r.multi_adj_prevalence_not_highest AS prevalence_not_highest, 
           r.multi_adj_prevalence_highest AS prevalence_highest, 
           r.multi_adj_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.multi_adj_positive_prevalence_highest AS positive_prevalence_highest, 
           r.multi_adj_positive_rate_not_highest AS positive_rate_not_highest, 
           r.multi_adj_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN base_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time
    
    
        UNION ALL

    -- Model selection on last time after multiple adjustment, with multiple adjustment applied
    SELECT 'Best Multi Adjusted Metric - MultiAdjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.multi_adj_value AS value,
           r.multi_adj_max_recall_ratio AS max_recall_ratio,
           r.multi_adj_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.multi_adj_frac_not_highest AS frac_not_highest,
           r.multi_adj_frac_highest AS frac_highest, 
           r.multi_adj_min_score_not_highest AS min_score_not_highest, 
           r.multi_adj_min_score_highest AS min_score_highest, 
           r.multi_adj_mean_score_not_highest AS mean_score_not_highest, 
           r.multi_adj_mean_score_highest AS mean_score_highest, 
           r.multi_adj_precision_not_highest AS precision_not_highest, 
           r.multi_adj_precision_highest AS precision_highest, 
           r.multi_adj_prevalence_not_highest AS prevalence_not_highest, 
           r.multi_adj_prevalence_highest AS prevalence_highest, 
           r.multi_adj_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.multi_adj_positive_prevalence_highest AS positive_prevalence_highest, 
           r.multi_adj_positive_rate_not_highest AS positive_rate_not_highest, 
           r.multi_adj_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN multi_adj_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time
    
    UNION ALL

    -- Model selection on last time after multiple adjustment, with multiple adjustment applied
    SELECT 'Best Adjusted Metric - MultiAdjusted'::VARCHAR(128) AS strategy,
           r.train_end_time, r.past_train_end_time,
           r.list_size, r.metric, r.parameter,
           r.multi_adj_value AS value,
           r.multi_adj_max_recall_ratio AS max_recall_ratio,
           r.multi_adj_recall_not_highest_to_highest AS recall_not_highest_to_highest,
           r.multi_adj_frac_not_highest AS frac_not_highest,
           r.multi_adj_frac_highest AS frac_highest, 
           r.multi_adj_min_score_not_highest AS min_score_not_highest, 
           r.multi_adj_min_score_highest AS min_score_highest, 
           r.multi_adj_mean_score_not_highest AS mean_score_not_highest, 
           r.multi_adj_mean_score_highest AS mean_score_highest, 
           r.multi_adj_precision_not_highest AS precision_not_highest, 
           r.multi_adj_precision_highest AS precision_highest, 
           r.multi_adj_prevalence_not_highest AS prevalence_not_highest, 
           r.multi_adj_prevalence_highest AS prevalence_highest, 
           r.multi_adj_positive_prevalence_not_highest AS positive_prevalence_not_highest, 
           r.multi_adj_positive_prevalence_highest AS positive_prevalence_highest, 
           r.multi_adj_positive_rate_not_highest AS positive_rate_not_highest, 
           r.multi_adj_positive_rate_highest AS positive_rate_highest
    FROM {schema}.model_adjustment_results_plevel r
    JOIN adj_mgs b
      ON r.model_group_id = b.model_group_id
      AND r.past_train_end_time = b.train_end_time
      AND r.list_size = b.list_size
      AND r.metric = b.metric
      AND r.parameter = b.parameter
    WHERE r.train_end_time > r.past_train_end_time

    UNION ALL

    -- Composite model
    SELECT 'Composite Model - Adjusted'::VARCHAR(128) AS strategy,
          train_end_time, past_train_end_time,
          list_size, metric, parameter,
          value,
          max_recall_ratio,
          recall_not_highest_to_highest,
          frac_not_highest,
          frac_highest, 
          min_score_not_highest, 
          min_score_highest, 
          mean_score_not_highest, 
          mean_score_highest, 
          precision_not_highest, 
          precision_highest, 
          prevalence_not_highest, 
          prevalence_highest, 
          positive_prevalence_not_highest, 
          positive_prevalence_highest, 
          positive_rate_not_highest, 
          positive_rate_highest
    FROM {schema}.composite_results_plevel
    WHERE train_end_time > past_train_end_time
    ;
    """.format(schema=schema)

    ts_df = pd.read_sql(sql, engine)

    ts_df['dataset'] = 'Education Crowdfunding'

    ts_df['strategy'] = ts_df['strategy'].map({
        'Best Unadjusted Metric - Unadjusted': 'Unmitigated', 
        'Best Adjusted Metric - Adjusted': 'Mitigated - Single Model', 
        'Best Adjusted Metric - MultiAdjusted': 'MultiMitigated - Adj Seln.',
        'Best Multi Adjusted Metric - MultiAdjusted': 'MultiMitigated - MAdj Seln.',
        'Composite Model - Adjusted': 'Mitigated - Composite Model',
        'Best Unadjusted Metric - Adjusted': 'Mitigated - Unadj. Model Seln.', 
        'Best Unadjusted Metric - MultiAdjusted': "MultiMitigated - Unadj. Model Seln."        
    })
    
    return ts_df

In [ ]:
def get_exhaustive_df(engine, schema="bias_results"):
    sql = f"""
    SELECT model_id, model_group_id , train_end_time , past_train_end_time , multi_adj_past_train_end_time , base_frac_highest , adj_frac_highest , multi_adj_frac_highest, base_recall_not_highest_to_highest , adj_recall_not_highest_to_highest , multi_adj_recall_not_highest_to_highest, base_value , adj_value , multi_adj_value  FROM {schema}.model_adjustment_results_plevel marp
    """
    df = pd.read_sql(sql, engine)
    return df
    

## Reproducing Figures from the Study

As a starting point, we can reproduce the figures from the study directly using the data in the `bias_results_submitted` schema:

In [ ]:
ts_df3 = get_bias_results(engine_donors, 'bias_results')
print(ts_df3.columns)

In [ ]:
import matplotlib.dates as mdates

months = mdates.MonthLocator([1,3,5,7,9,11])
months_fmt = mdates.DateFormatter('%b %y')

In [ ]:
def line_plot_over_time(y, y_label, ax=None, strategies=["Unmitigated", "Mitigated - Single Model", "MultiMitigated - Adj Seln.", "MultiMitigated - MAdj Seln.", "MultiMitigated - Unadj. Model Seln."]):
    if strategies is None:
        df = ts_df3
    else:
        df = ts_df3.loc[ts_df3["strategy"].isin(strategies), ]
    if len(strategies) == 1:
        hue = None
    else:
        hue = "strategy"
    if ax is None:
        fig, ax = plt.subplots(figsize=(12,5))
    sns.lineplot(data=df, x='train_end_time', y=y, hue=hue, marker="o", ax=ax)

    ax.set_ylabel(y_label, fontsize=14)
    ax.set_xlabel('Test Set Date', fontsize=14)

    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(months_fmt)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)

    ax.get_legend()
    return ax

In [ ]:
def line_plots_over_time(ys, y_labels, ax=None, strategies=["Unmitigated", "Mitigated - Single Model"]):
    if strategies is None:
        df = ts_df3
    else:
        df = ts_df3.loc[ts_df3["strategy"].isin(strategies), ]    
    if len(strategies) == 1:
        hue = None
    else:
        hue = "strategy"
    markers = ["*", "o", "s", "^", "P"]
    if ax is None:
        fig, ax = plt.subplots(figsize=(12,5))
    for i, y in enumerate(ys):
        y_label = y_labels[i]
        marker = markers[i]
        sns.lineplot(data=df, x='train_end_time', y=y, hue=hue, marker=marker, ax=ax, label=y_label)
    ax.set_xlabel('Test Set Date', fontsize=14)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(months_fmt)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    
    ax.get_legend()
    
    return ax

### Supplementary Figure 3
Comparing model precision (a) and disparity (b) metrics over time fordifferent model selection strategies in the Education Crowdfunding policy context.

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=ts_df3.loc[ts_df3['strategy'] != 'Mitigated - Unadj. Model Seln.', ], x='train_end_time', y='value', hue='strategy', marker="o", ax=ax)

ax.set_ylim((0.30,0.70))
ax.set_ylabel('Overall Precision at 1000', fontsize=14)
ax.set_xlabel('')

ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(months_fmt)

ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)

handles, labels = ax.get_legend_handles_labels()
hhandles = []
llabels = []
for i, lab in enumerate(labels):
    if lab not in ('strategy'):
        llabels.append(lab)
        hhandles.append(handles[i])
ax.legend(hhandles, llabels, ncol=1, fontsize=14, loc='lower left')

In [ ]:
line_plot_over_time(y="recall_not_highest_to_highest", y_label='Recall Disparity:\nLower to Higher Poverty')

In [ ]:
line_plot_over_time(y="frac_highest", y_label='Fraction of list set to highest plevel')

In [ ]:
df = ts_df3.copy()[~ts_df3["strategy"].isin(["Unmitigated"])]
grouped = df.groupby("strategy")
mean_val = grouped.mean()["value"] 
std_val = grouped.std()["value"]
c_mean_recp = grouped.mean()["recall_not_highest_to_highest"] - 1
std_recp = grouped.std()["recall_not_highest_to_highest"]
mean_val.plot(kind="bar")
plt.title("Mean Precision@1000")

In [ ]:
c_mean_recp.plot(kind="bar")
plt.title("Mean Recall Parity - 1")

In [ ]:
std_recp.plot(kind="bar", )
plt.title("Stdev Recall Parity")

In [ ]:
df["rpt"] = abs(df["recall_not_highest_to_highest"] - 1)
for strat in list(df["strategy"].unique()):
    df[df["strategy"] == strat]["rpt"].plot(kind="hist")
    plt.title(f"Distribution of Abs(Recall Parity-1) for {strat}")
    plt.show()

### Figure 2 (Education Crowdfunding Points)
Comparing trade-offs across strategies

In [ ]:
# renaming a few columns to ensure consistency across datasets
comp_df = ts_df3.rename(
    {'recall_not_highest_to_highest': 'recall_disp', 
     'frac_not_highest': 'frac_grp1', 
     'frac_highest': 'frac_grp2'
    }, axis='columns')

In [ ]:
sty_ord = [
    'Mitigated - Single Model', 
    'Mitigated - Composite Model',
    'Mitigated - Unadj. Model Seln.',
    'MultiMitigated - Adj Seln.', 
    'Unmitigated'
]

fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=comp_df.groupby(['dataset', 'strategy'])[['value', 'recall_disp']].mean().reset_index(), 
              x='value', y='recall_disp', 
              hue='dataset', 
              style='strategy',
              style_order=sty_ord,
              markers=['X', 'o', 'P', 's'],
              markersize=10,
              dashes=None,
              ci=None,
              ax=ax)


for i, ds in enumerate(comp_df['dataset'].unique()):
    tmp_df = comp_df.loc[comp_df['dataset'] == ds, ].copy()
    
    x_coords = list(tmp_df.groupby(['dataset', 'strategy'])[['value', 'recall_disp']].mean().reset_index()['value'].values)
    y_coords = list(tmp_df.groupby(['dataset', 'strategy'])[['value', 'recall_disp']].mean().reset_index()['recall_disp'].values)

    prec_errors = 1.96*tmp_df.groupby(['dataset', 'strategy'])['value'].sem().values
    disp_errors = 1.96*tmp_df.groupby(['dataset', 'strategy'])['recall_disp'].sem().values
    colors = sns.color_palette().as_hex()[i]
    ax.errorbar(x_coords, y_coords, 
                xerr=prec_errors, 
                yerr=disp_errors,
        ecolor=colors, 
        fmt=' ', zorder=-1, capsize=5)


ax.set_xlim((0.40,0.82))

ax.set_ylabel('Recall Disparity', fontsize=16)
ax.set_xlabel('Precision at Top k', fontsize=16)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

handles, labels = ax.get_legend_handles_labels()
hhandles = []
llabels = []
for i, lab in enumerate(labels):
    if lab not in list(comp_df['dataset'].unique()) + ['dataset', 'strategy']:
        handles[i].set_linestyle("")
    hhandles.append(handles[i])
    llabels.append(lab)
ax.legend(hhandles, llabels, fontsize=16, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., markerscale=2)

fig.tight_layout()

## Reproducing the Bias Adjustments

The following code will re-run the bias analysis itself starting from the model results. Note that there is some randomness in the code (breaking ties between projects with the same score as well as breaking ties between models with the same performance on a given test set), so the results you get here may not be exactly identical to the results above, but should be statistically consistent with them.

**NOTE: Running all of the results here can take a fair amount of time (around 2-3 hours on a db.m5.2xlarge AWS RDS instance), so be sure to be working in a screen/tmux setting if running the notebook remotely!**

Currently, this code assumes your data is in a postgres database containing modeling results in the format of our open-source `triage` machine learning pipeline, including temporal validation splits.


Next, let's clear out anything in the `bias_results` schema (these should start empty, unless you've already run anything since loading the database):

In [ ]:
engine_donors.execute('TRUNCATE TABLE bias_results.composite_results_plevel;')
engine_donors.execute('TRUNCATE TABLE bias_results.model_adjustment_results_plevel;')
engine_donors.execute('TRUNCATE TABLE bias_working.model_adjustment_group_k_plevel;')
engine_donors.execute('COMMIT;')

In [ ]:
from DJRecallAdjuster import education_ra_procedure

In [ ]:
weights = [0.99, 0.01]

In [ ]:
education_ra_procedure(weights=weights)

If you want to explore the results further, `bias_results.model_adjustment_results_plevel` holds the results from your run for each model, including the validation set the thresholds were learned based on (`past_train_end_time`), the validation set they were applied to (`train_end_time`), and the unadjusted (`base_` columns) and adjusted (`adj_` columns) results in terms of accuracy, disparities, and the composition of the selected list. The `bias_results.composite_results_plevel` has a similar structure for the composite models created by selecting the best-performing model for each subgroup separately (so note there's no overall `model_id` ehre, but separate columns for the model group used for each subgroup)



In [ ]:
pd.read_sql("SELECT * FROM bias_results.model_adjustment_results_plevel LIMIT 5", engine_donors)